# Access to data in the cloud (GCS)

In [1]:
import sys
import gcsfs
import xarray as xr
import intake

/Users/gmaze/anaconda/envs/obidam36/lib/python3.6/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['cmip5', 'spark_dataframe', 'spark_rdd', 'spark_cat'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


## Read data from Google Cloud Storage (gcsfs)

### Access and listing

In [ ]:
# Define cloud file system access point:
fs = gcsfs.GCSFileSystem(project='argo-france', token='anon', access='read_only')

# And list content of a bucket:
fs.ls('argodata')

But data access with ``gcsfs`` is critically dependant on the GCS set-up. For instance the following project does not allow to list the bucket content:

In [ ]:
fs2 = gcsfs.GCSFileSystem(project='alert-ground-261008', token='anon', access='read_only')
try:
    fs2.ls('data_bdo2020')
except:
    print(sys.exc_info()[0])

On the other hand, some dataset may not be free and use a requester pay model. 
In this case, you would have to properly manage authentication:

In [ ]:
fs3 = gcsfs.GCSFileSystem(project='poised-honor-358', token='anon')
try:
    fs3.ls('somovar-02')
except ValueError as e:
    print(str(e))

### Load data

In [ ]:
gcsmap = fs.get_mapper("argodata/gridded/ISAS15_TEMP_NATL.zarr")
ds = xr.open_zarr(gcsmap)
print("Size of the dataset:", ds.nbytes/1e9,"Gb")
print(ds)

In [ ]:
# Load another dataset:
# gcsmap = fs.get_mapper('argodata/sdl/GLOBAL_ARGO_SDL2000')
gcsmap = fs.get_mapper('argodata/sdl/GLOB_HOMOGENEOUS_variables.zarr')
ds = xr.open_zarr(gcsmap)
print("Size of the dataset:", ds.nbytes/1e9,"Gb")
print(ds)

## Use intake catalog of data

The catalog also uses the gcsfs entry point, but with intake it's transparent to the user:

### Access and listing of the catalog

In [ ]:
catalog_url = 'https://raw.githubusercontent.com/obidam/ds2-2020/master/ds2_data_catalog.yml'
cat = intake.Catalog(catalog_url)
list(cat)

### Load data

In [ ]:
ds = cat.en4.read_chunked()
ds

# Pangeo data

[Note that Pangeo is currently re-organising its datastore](https://discourse.pangeo.io/t/cleaning-out-the-pangeo-data-google-cloud-storage-bucket/353/22).

Some dataset access points may thus be obsolete below...

## Direct access to GCS

In [ ]:
# Define cloud file system access point:
fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='anon', access='read_only')
fs.ls('pangeo-data')

In [ ]:
### Load a dataset:
# gcsmap = gcsfs.mapping.GCSMap('pangeo-data/dataset-duacs-rep-global-merged-allsat-phy-l4-v3-alt', gcs=fs) # AVISO altimetry
# gcsmap = gcsfs.mapping.GCSMap('pangeo-data/eNATL60-BLBT02X-ssh', gcs=fs) # High Resolution North Atlantic Ocean simulation
gcsmap = gcsfs.mapping.GCSMap('pangeo-era5/reanalysis/spatial-analysis', gcs=fs) # Hourly Atmospheric fields (1979/2018)
ds = xr.open_zarr(gcsmap)
print("Size of the dataset:", ds.nbytes/1e9,"Gb")
print(ds)

## Intake Pangeo catalog

In [2]:
catalog_url = 'https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml'
cat = intake.Catalog(catalog_url)
print(list(cat))
print(list(cat.ocean))
print(list(cat.atmosphere))
print(list(cat.hydro))
# cat.walk(depth=5)

['ocean', 'atmosphere', 'hydro']
['sea_surface_height', 'cesm_mom6_example', 'ECCOv4r3', 'SOSE', 'GODAS', 'ECCO_layers', 'LLC4320', 'GFDL_CM2_6', 'CESM_POP', 'channel']
['gmet_v1', 'trmm_3b42rt', 'sam_ngaqua_qobs_eqx_3d', 'sam_ngaqua_qobs_eqx_2d', 'gpcp_cdr_daily_v1_3', 'wrf50_erai']
['cgiar_pet', 'hydrosheds_dir', 'hydrosheds_acc', 'hydrosheds_dem']


In [3]:
# ds = cat.ocean.SOSE.read_chunked()
# ds = cat.atmosphere.gmet_v1.read_chunked()
ds = cat.ocean.sea_surface_height.read_chunked()
ds

/Users/gmaze/anaconda/envs/obidam36/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
_call out of retries on exception: ('invalid_grant: Bad Request', '{\n  "error": "invalid_grant",\n  "error_description": "Bad Request"\n}')
Traceback (most recent call last):
  File "/Users/gmaze/anaconda/envs/obidam36/lib/python3.6/site-packages/gcsfs/core.py", line 534, in _call
    timeout=self.requests_timeout,
  File "/Users/gmaze/anaconda/envs/obidam36/lib/python3.6/site-packages/google/auth/transport/requests.py", line 212, in request
    self

KeyError: 'pangeo-cmems-duacs/.zmetadata'

## Access to data subset

In [ ]:
# Load and plot a map
%matplotlib inline
%time sla = ds['adt'].sel(time='2009-02-12', method='nearest')
print(sla)
sla.plot()

In [ ]:
# Load and plot a time series
# (this slicing is much longer !)
%time sla = ds['adt'].sel(latitude=30, method='nearest').sel(longitude=360-55, method='nearest')
print(sla)
sla.plot()